<a href="https://colab.research.google.com/github/PranathiDoddipalli/Python_DSA/blob/main/data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSA 5200 Exercise 3 for Module 05 - 09

In this exercise, you will use [the air quality data in India](https://www.kaggle.com/datasets/shrutibhargava94/india-air-quality-data).

In [ ]:
import io  # Input/Output
import zipfile  # Help read file from Zip file without decompressing files
from pathlib import Path  # Objective filesystem paths management

import numpy as np  # Numpy array algebra and statistics
import pandas as pd  # Pandas dataframe

# Notebook magics to automatically reload libraries
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Load the data
cdir = Path("./")  # Current directory

with zipfile.ZipFile(cdir / "/india_air_quality_data.zip", mode="r") as zipobj:
    # File name
    zipinfos = [info for info in zipobj.infolist() if info.filename.endswith("csv")]
    print(zipinfos[0].filename)

    # Read data
    india_aq = pd.read_csv(
        zipobj.open(zipinfos[0].filename), encoding="iso8859-1", parse_dates=["date"]
    )

data.csv


<ipython-input-29-da09b0202fcc>:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


Once we load the data, it is necessary to understand the general structure of it.


In [ ]:
india_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435742 entries, 0 to 435741
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   stn_code                     291665 non-null  object        
 1   sampling_date                435739 non-null  object        
 2   state                        435742 non-null  object        
 3   location                     435739 non-null  object        
 4   agency                       286261 non-null  object        
 5   type                         430349 non-null  object        
 6   so2                          401096 non-null  float64       
 7   no2                          419509 non-null  float64       
 8   rspm                         395520 non-null  float64       
 9   spm                          198355 non-null  float64       
 10  location_monitoring_station  408251 non-null  object        
 11  pm2_5                     

In [ ]:
# Sample statistics to help do exploratory analysis
india_aq.describe()

,so2,no2,rspm,spm,pm2_5,date
count,401096.000000,419509.000000,395520.000000,198355.000000,9314.000000,435735
mean,10.829414,25.809623,108.832784,220.783480,40.791467,2010-01-11 07:22:01.301249792
min,0.000000,0.000000,0.000000,0.000000,3.000000,1987-01-01 00:00:00
25%,5.000000,14.000000,56.000000,111.000000,24.000000,2007-07-03 00:00:00
50%,8.000000,22.000000,90.000000,187.000000,32.000000,2010-11-12 00:00:00
75%,13.700000,32.200000,142.000000,296.000000,46.000000,2013-09-07 12:00:00
max,909.000000,876.000000,6307.033333,3380.000000,504.000000,2015-12-31 00:00:00
std,11.177187,18.503086,74.872430,151.395457,30.832525,NaN


The data set is well structured and has already completed the basic data carpentry process. The original source of the dataset was [the India historical daily ambient air quality data](https://data.gov.in/catalog/historical-daily-ambient-air-quality-data), which was collected and first published in 2016. Let's take a look at what the data tells us by using the conceptual model. Unfortunately, it is impossible to find the metadata or description of the historical daily ambient air quality data. However, it is possible to understand its conceptual model by using [the Ambient Air Quality Data of Delhi Stations for the Month of June 2018](https://cpcb.nic.in/displaypdf.php?id=bWFudWFsLW1vbml0b3JpbmcvQWlyX1F1YWxpdHlfRGF0YV9EZWxoaS1OQ1JfSnVuZTIwMTgucGRm)

| Variable Notation | Description | Unit | Daily Average Limit of pollutions |
| -- | -- | -- | -- |
| stn_code | Subscriber Trunk Dialing Code or Area Code | - | - |
| sampling_date | Data collecting date | [Month - M]MMYYYY | - |
| state         | State name           | - | - |
| location      | Ambient Air Quality Monitoring (AAQM) and National Air Quality Monitoring (NAMP) Stations' location | - | - |
| agency        | Agency or department of collecting data | - | - |
| type          | Data collecting area type | Sensitive Area, Industrial Area, Residential, etc. | - |
| so2 | Sulfur dioxide | $\mu\text{g/m}^3$ | $80\mu\text{g/m}^3$ |
| no2 | Nitrogen dioxide | $\mu\text{g/m}^3$ | $80\mu\text{g/m}^3$ |
| rspm | Respiratory Suspended Particulate Matter (smaller than 10 micrometers) | $\mu\text{g/m}^3$ | $100\mu\text{g/m}^3$ |
| spm | Suspended Particulate Matter (smaller than 50-100 micrometers) | $\mu\text{g/m}^3$ | - |
| location_monitoring_station | Similar to `location` | - | - |
| pm2_5 | Fina Particulate Matter (smaller than 2.5 micrometers) | $\mu\text{g/m}^3$ | $60\mu\text{g/m}^3$ |
| date | - | - | YYYY-MM-DD |

According to the Delhi stations' report, there are no explicit definitions of respiratory suspended particulate matter and suspended particulate matter, so those were presumed by using the general definition of those two based on [the European Environment Agency](https://www.eea.europa.eu/publications/2-9167-057-X/page021.html). In addition to the lack of precise definition of each variable, some variable notation seems to be duplicated. For example, `location` and `location_monitoring_station` are highly likely the same columns to indicate the data sampling or monitory station. **Without the explicit data conceptual model description, data scientists have difficulty in exploring the data itself and later use it for training and building appropriate models**.   

---

**Activity 1**. The data set has some issues, such as the duplicate columns. Please subset an original dataset. (1) **spatial information**, (2) **time or date**, and (3) **air quality measures** have to keep those for the later analysis. Also, clear the (4) **observation rows that include not available or empty values**.
What are your criteria for dropping NA values? Do we need to drop an observation row if at least one air quality measure (`so2`, `no2`, `rspm`, `spm`, and `pm2_5`) is empty? Or should we delete a row when all air quality measures are not available? Please give your opinion on (5) **your criteria to drop observations**.



In [ ]:
# 1) subset of original dataset as mentioned as spatial info # 2)time or date and # 3)air quality measures

columns_to_keep = ['location', 'state', 'sampling_date', 'so2', 'no2', 'rspm', 'spm', 'pm2_5']


df = india_aq[columns_to_keep]
df.head()

,location,state,sampling_date,so2,no2,rspm,spm,pm2_5
0,Hyderabad,Andhra Pradesh,February - M021990,4.8,17.4,NaN,NaN,NaN
1,Hyderabad,Andhra Pradesh,February - M021990,3.1,7.0,NaN,NaN,NaN
2,Hyderabad,Andhra Pradesh,February - M021990,6.2,28.5,NaN,NaN,NaN
3,Hyderabad,Andhra Pradesh,March - M031990,6.3,14.7,NaN,NaN,NaN
4,Hyderabad,Andhra Pradesh,March - M031990,4.7,7.5,NaN,NaN,NaN


In [ ]:
#4)  observation rows that include not available or empty values.

# Calculate the percentage of null values in each column
null_percentages = df.isnull().mean() * 100

# Print the result
print(null_percentages)



location          0.000688
state             0.000000
sampling_date     0.000688
so2               7.951035
no2               3.725370
rspm              9.230692
spm              54.478797
pm2_5            97.862497
dtype: float64


**Answer your criteria to drop observations:**

1) location, state, sampling_date: These columns have very low percentages of null values, so they should definitely be kept. The missing values can be filled with the mode (most frequent value) of the column.

2) so2, no2, rspm: These columns have a relatively low percentage of null values. They can be kept and the missing values can be filled with the mean or median of the column. The choice between mean and median depends on the distribution of the data. If the data is skewed, median is a better choice. will find it out in next steps the skewness.

3) spm: This column has a higher percentage of null values. If this column is important for your analysis, you could fill the missing values using a method that makes sense in your context (mean, median, mode, or even using a predictive model). If it’s not critical, you might consider dropping it.

4) pm2_5: This column has a very high percentage of null values. Generally, it’s recommended to drop columns where a large majority of values are missing, as imputing them might lead to biased results. So dropping this column.

Handling Missing Values depends on the nature of analysis and the amount of missing data.
Here are two common strategies:

**Drop Rows with Any Missing Value:** If an observation has at least one air quality measure (so2, no2, rspm, spm, and pm2_5) that is empty,
                                    you drop that entire observation. This method is simple and ensures that all data is complete for all measures.
                                    However, it might lead to loss of valuable data if the dataset has a lot of missing values. Therefore using median to fill null values for **so2, no2,                                         rspm** and used mode for **location, state, sampling_date.**

**Drop Rows with All Missing Values:** If an observation has all air quality measures missing, only then you drop that observation.
                                        This method retains more data, therefore
                                        dropped **pm2_5**

In [ ]:
columns_to_fill = ['location', 'state', 'sampling_date']

# Fill missing values with the mode of each column
for column in columns_to_fill:
    df.loc[:, column] = df.loc[:, column].fillna(df[column].mode()[0])
df.head()

,location,state,sampling_date,so2,no2,rspm,spm,pm2_5
0,Hyderabad,Andhra Pradesh,February - M021990,4.8,17.4,NaN,NaN,NaN
1,Hyderabad,Andhra Pradesh,February - M021990,3.1,7.0,NaN,NaN,NaN
2,Hyderabad,Andhra Pradesh,February - M021990,6.2,28.5,NaN,NaN,NaN
3,Hyderabad,Andhra Pradesh,March - M031990,6.3,14.7,NaN,NaN,NaN
4,Hyderabad,Andhra Pradesh,March - M031990,4.7,7.5,NaN,NaN,NaN


In [ ]:
# Calculate the correlation of 'spm' with other columns
numeric_cols = df.select_dtypes(include=[np.number])
correlation = numeric_cols.corr()['spm']
print(correlation)


so2      0.148325
no2      0.326170
rspm     0.801752
spm      1.000000
pm2_5         NaN
Name: spm, dtype: float64


In [ ]:
# Select only the numeric columns in the DataFrame
numeric_cols = df.select_dtypes(include=[np.number])

skewness = numeric_cols.skew()

print(skewness)

so2      8.521066
no2      3.676816
rspm     3.213677
spm      1.585091
pm2_5    3.352237
dtype: float64


Given the skewness and null percentages of your data,

1. **so2 (Skewness: 8.521066, Null Percentage: 7.951035%)**: Since this column is highly skewed and has a moderate percentage of null values, fill the missing values with the median, as the mean would be influenced by the skewness.

2. **no2 (Skewness: 3.676816, Null Percentage: 3.725370%)**: This column is moderately skewed and has a relatively low percentage of null values. Filling the missing values with the median would be a good choice here as well.

3. **rspm (Skewness: 3.213677, Null Percentage: 9.230692%)**: Similar to 'no2', this column is moderately skewed and has a moderate percentage of null values. The median would be a suitable choice for filling the missing values.

4. **spm (Skewness: 1.585091, Null Percentage: 54.478797%)**: This column is slightly skewed and has a high percentage of null values. Given the high percentage of missing values,using a predictive model to estimate the missing values based on the other columns( or can use median if dataset .

5. **pm2_5 (Skewness: 3.352237, Null Percentage: 97.862497%)**: This column has a very high percentage of null values and is moderately skewed. Given that almost all values are missing, it might be best to drop this column, as imputing such a large number of values could lead to biased results.


In [ ]:
df = df.drop(columns=['pm2_5'])

In [ ]:
def fill_na_with_median(column):
    median = column.median()
    column.fillna(median, inplace=True)

# Apply the function to the 'so2', 'no2', 'rspm'
for column in ['so2', 'no2', 'rspm']:
    fill_na_with_median(df[column])
df.head()

,location,state,sampling_date,so2,no2,rspm,spm
0,Hyderabad,Andhra Pradesh,February - M021990,4.8,17.4,90.0,NaN
1,Hyderabad,Andhra Pradesh,February - M021990,3.1,7.0,90.0,NaN
2,Hyderabad,Andhra Pradesh,February - M021990,6.2,28.5,90.0,NaN
3,Hyderabad,Andhra Pradesh,March - M031990,6.3,14.7,90.0,NaN
4,Hyderabad,Andhra Pradesh,March - M031990,4.7,7.5,90.0,NaN


As correlation of spm with other values is +ve we use a knn predictive model (using subset, because the main dataset is taking so long time) to fill null values

In [ ]:
from sklearn.impute import KNNImputer

# Separate the 'spm' column
spm_col = df[['spm']]

# Take a subset of the data for faster computation
subset_spm_col = spm_col.sample(frac=0.1, random_state=1)

imputer = KNNImputer(n_neighbors=5)
subset_spm_imputed = imputer.fit_transform(subset_spm_col)
subset_spm_imputed = pd.DataFrame(subset_spm_imputed, columns=['spm'])

df = df.drop(columns=['spm'])
df = pd.concat([df, subset_spm_imputed], axis=1)
df.head()


,location,state,sampling_date,so2,no2,rspm,spm
0,Hyderabad,Andhra Pradesh,February - M021990,4.8,17.4,90.0,75.000000
1,Hyderabad,Andhra Pradesh,February - M021990,3.1,7.0,90.0,188.666667
2,Hyderabad,Andhra Pradesh,February - M021990,6.2,28.5,90.0,219.629107
3,Hyderabad,Andhra Pradesh,March - M031990,6.3,14.7,90.0,71.000000
4,Hyderabad,Andhra Pradesh,March - M031990,4.7,7.5,90.0,416.000000


In [ ]:
import plotnine as pn
from plotnine import *

**Activity 2.** Now, let's focus on sulfur dioxide (`so2`), first. It is a great idea to confirm whether anomalies (or outliers) exist by each region before drawing charts to find meaninful insight. **Create the descriptive table by each state**.

In [ ]:
# Write your codes below
# ------------------------------------
descriptive_table = df.groupby('state')['so2'].describe()
print(descriptive_table)

                               count       mean        std   min        25%  \
state                                                                         
Andhra Pradesh               26368.0   7.303478   6.456558   0.9   4.000000   
Arunachal Pradesh               90.0   4.411111   2.863542   1.0   2.000000   
Assam                        19361.0   6.732033   2.407549   0.4   5.300000   
Bihar                         2275.0  18.641055  18.338031   2.0   8.000000   
Chandigarh                    8520.0   5.239155   3.244834   0.7   2.000000   
Chhattisgarh                  7831.0  12.535921   4.972868   2.0  10.000000   
Dadra & Nagar Haveli           634.0   8.932177   2.561510   4.5   7.400000   
Daman & Diu                    782.0   8.192711   3.077823   1.8   7.000000   
Delhi                         8551.0   8.673383   7.278709   0.5   4.000000   
Goa                           6206.0   7.029053   7.103367   0.0   4.000000   
Gujarat                      21279.0  16.619324  10.

**Activity 3**. If you saw the difference between the `75% quartile` and `maximum` values in the above table you pointed out, you can realize that one state has an extremely large difference between two statistics. **Please find (1) what state has a suspicious anomaly based on the distance between `75% quartile` and `maximum` values.**

**Write your answer:**

Based on the table you provided, the state with the largest difference between the 75% quartile and the maximum value for ‘so2’ is **Tamil Nadu.**
The 75% quartile value for ‘so2’ in Tamil Nadu is **14**, while the maximum value is **909**.

This is a significant difference and suggests a potential anomaly or outlier in the ‘so2’ measurements for Tamil Nadu.

**Activity 4**. Based on your answer in **Activity 3**, please drop a single row containing the extreme `so2` observation.


In [ ]:
# Write your code below
# -------------------------
# Find the index of the row with the maximum 'so2' value for Tamil Nadu
index_to_drop = df[(df['state'] == 'Tamil Nadu') & (df['so2'] == df[df['state'] == 'Tamil Nadu']['so2'].max())].index

df = df.drop(index_to_drop)
df.head()

,location,state,sampling_date,so2,no2,rspm,spm
0,Hyderabad,Andhra Pradesh,February - M021990,4.8,17.4,90.0,75.000000
1,Hyderabad,Andhra Pradesh,February - M021990,3.1,7.0,90.0,188.666667
2,Hyderabad,Andhra Pradesh,February - M021990,6.2,28.5,90.0,219.629107
3,Hyderabad,Andhra Pradesh,March - M031990,6.3,14.7,90.0,71.000000
4,Hyderabad,Andhra Pradesh,March - M031990,4.7,7.5,90.0,416.000000


Next, please take a look at the head and tail of your subset. The `date` column indicates that the observations are irregularly collected by time. Some are monthly, but others are daily.

In [ ]:
subset_df2 = df
display(subset_df2.head())
display(subset_df2.tail())

,location,state,sampling_date,so2,no2,rspm,spm
0,Hyderabad,Andhra Pradesh,February - M021990,4.8,17.4,90.0,75.000000
1,Hyderabad,Andhra Pradesh,February - M021990,3.1,7.0,90.0,188.666667
2,Hyderabad,Andhra Pradesh,February - M021990,6.2,28.5,90.0,219.629107
3,Hyderabad,Andhra Pradesh,March - M031990,6.3,14.7,90.0,71.000000
4,Hyderabad,Andhra Pradesh,March - M031990,4.7,7.5,90.0,416.000000


,location,state,sampling_date,so2,no2,rspm,spm
435737,ULUBERIA,West Bengal,24-12-15,22.0,50.0,143.0,NaN
435738,ULUBERIA,West Bengal,29-12-15,20.0,46.0,171.0,NaN
435739,Guwahati,andaman-and-nicobar-islands,19-03-15,8.0,22.0,90.0,NaN
435740,Guwahati,Lakshadweep,19-03-15,8.0,22.0,90.0,NaN
435741,Guwahati,Tripura,19-03-15,8.0,22.0,90.0,NaN


**Activity 5**. To amend this irregular time period, let's create the monthly average of air quality measures. It is a great idea to follow the following steps to achieve this:

1. Create `year` and `month` columns from the `date` column
2. Create the monthly average values by using the `.groupby` and `.agg` methods
3. Create or update the `date` column with `datetime64[ns]` data type from the `year` and `month` columns
4. Stack all air quality measuring values (`so2`, `no2`, `rspm`, `spm`, `pm2_5`) to make a long-form data.

In [ ]:
# Write your code below
# --------------------------------------

df['sampling_date'] = pd.to_datetime(df['sampling_date'], errors='coerce')
subset_df2['year'] = subset_df2['sampling_date'].dt.year
subset_df2['month'] = subset_df2['sampling_date'].dt.month

# Create the monthly average values using the .groupby and .agg methods
monthly_averages = subset_df2.groupby(['state', 'year', 'month']).agg({'so2': 'mean', 'no2': 'mean', 'rspm': 'mean', 'spm': 'mean'}).reset_index()

# Create or update the date column with datetime64[ns] data type from the year and month columns
monthly_averages['date'] = pd.to_datetime(monthly_averages[['year', 'month']].assign(day=1))

# Stack all air quality measuring values to make a long-form data
long_form_data = monthly_averages.melt(id_vars=['state', 'date'], value_vars=['so2', 'no2', 'rspm', 'spm'], var_name='measure', value_name='value')

long_form_data.head()



<ipython-input-44-3c7eb1c99435>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.


,state,date,measure,value
0,Andhra Pradesh,2004-01-01,so2,7.933333
1,Andhra Pradesh,2004-02-01,so2,7.577778
2,Andhra Pradesh,2004-03-01,so2,7.707018
3,Andhra Pradesh,2004-04-01,so2,5.912281
4,Andhra Pradesh,2004-05-01,so2,6.754237


**Activity 6**. Create **the monthly line charts of `so2` and `no2` for each regions**, using the data reshaped in **Activity 5**.

In [ ]:
# Resize the plot size
pn.options.set_option("figure_size", (16.0, 20.0))

(16.0, 20.0)

In [ ]:
# Write your code here
# -------------------------
from plotnine import options
pn.options.figure_size = (16.0, 20.0)
lfd = long_form_data
# Convert the 'date' column to datetime format
lfd['date'] = pd.to_datetime(lfd['date'])
lfd.set_index('date', inplace=True)

lfd['month'] = lfd.index.month
monthly_df = lfd.groupby(['state', 'measure', 'month']).mean().reset_index()

# Filter for so2 and no2 measures
filtered_df = monthly_df[monthly_df['measure'].isin(['so2', 'no2'])]

# Create line charts for each region
for region in df['state'].unique():
    plot = (ggplot(filtered_df[filtered_df['state'] == region], aes(x='month', y='value', color='measure')) +
            geom_line() +
            facet_wrap('~state') +
            ggtitle(f'Monthly so2 and no2 levels in {region}'))
    plot.show()

AttributeError: 'ggplot' object has no attribute 'show'

Once you draw small multiples of time series line charts of `so2` and `no2`, you can find some periodical patterns. Most of the time, the time series data has periodical autocorrelation (or patterns) over time. For example, there are typical seasonalities in producing some products annually. To remove or smooth these patterns to confirm the stationary in time series. If you are interested in analyzing time series deeper, please think about the time series analysis in statsitics.

The simplest way to smoothing the time series in Python is to use the `.rolling` method with `.mean`. The method calculates the moving average. For example, a 12-month period moving window is calculated if you define `.rolling(12).mean()` in your monthly data.

In [ ]:

df['sampling_date'] = pd.to_datetime(df['sampling_date'], errors='coerce')
subset_df2['year'] = subset_df2['sampling_date'].dt.year
subset_df2['month'] = subset_df2['sampling_date'].dt.month

# Create the monthly average values using the .groupby and .agg methods
monthly_averages = subset_df2.groupby(['state', 'year', 'month']).agg({'so2': 'mean', 'no2': 'mean', 'rspm': 'mean', 'spm': 'mean'}).reset_index()

# Create or update the date column with datetime64[ns] data type from the year and month columns
monthly_averages['date'] = pd.to_datetime(monthly_averages[['year', 'month']].assign(day=1))

# Stack all air quality measuring values to make a long-form data
monthly = monthly_averages.melt(id_vars=['state', 'date'], value_vars=['so2', 'no2', 'rspm', 'spm'], var_name='measure', value_name='value')

monthly.head()

,state,date,measure,value
0,Andhra Pradesh,2004-01-01,so2,7.933333
1,Andhra Pradesh,2004-02-01,so2,7.577778
2,Andhra Pradesh,2004-03-01,so2,7.707018
3,Andhra Pradesh,2004-04-01,so2,5.912281
4,Andhra Pradesh,2004-05-01,so2,6.754237


In [ ]:
# Unstack monthly data and create a new table named 'smooth_monthly'
smooth_monthly = monthly.set_index(["state", "date", "measure"]).unstack()
# Adjust the column index from the MultiIndex to the single index of air quality measure variables
smooth_monthly.columns = smooth_monthly.columns.get_level_values("measure")

In [ ]:
print(smooth_monthly.columns)

Index(['no2', 'rspm', 'so2', 'spm'], dtype='object', name='measure')


**Activity 7**. Create the 12-month moving average from the above `smooth_monthly` DataFrame.

In [ ]:
# Write your code here
# ----------------------------

smooth_monthly = (
    smooth_monthly.groupby(
        [
            # Fill in which row index going to be used for the 'groupby' calculation
            'state'
        ]
    )
    # Write down the 12-month moving average method below
    .rolling(12).mean()

    # Add suffix of each column name
    .add_suffix("_smooth")
    # Stack columns again
    .stack()
    # Convert from the Series to DataFrame
    .to_frame("value")
)
# Rest of process
smooth_monthly.index = smooth_monthly.index.droplevel(level=0)
smooth_monthly.reset_index(inplace=True)

In [ ]:
smooth_monthly.head()

,state,date,measure,value
0,Andhra Pradesh,2004-12-01,no2_smooth,32.253380
1,Andhra Pradesh,2004-12-01,rspm_smooth,85.026803
2,Andhra Pradesh,2004-12-01,so2_smooth,7.316827
3,Andhra Pradesh,2004-12-01,spm_smooth,218.866519
4,Andhra Pradesh,2005-01-01,no2_smooth,32.211888


**Activity 8**. Redraw **(1) the monthly line charts of `so2` and `no2` for each region** and plot **(2) the monthly line charts of the rest of the air quality measure variables for each region**, using the smoothed data.


In [ ]:
# Filter for so2 and no2 measures
so2_no2_df = smooth_monthly[smooth_monthly['measure'].isin(['so2_smooth', 'no2_smooth'])]

# Filter for the rest of the air quality measures
other_measures_df = smooth_monthly[~smooth_monthly['measure'].isin(['so2_smooth', 'no2_smooth'])]

**(1) Answer your code for redrawing `so2` and `no2`**:

In [ ]:
# Write your code here
# -------------------------
for region in smooth_monthly['state'].unique():
    # Plot for so2 and no2
    plot = (ggplot(so2_no2_df[so2_no2_df['state'] == region], aes(x='date', y='value', color='measure')) +
            geom_line() +
            ggtitle(f'Monthly so2 and no2 levels in {region}'))
    plot.show()

AttributeError: 'ggplot' object has no attribute 'show'

**(2) Answer your code for plotting the rest of air quality variables, such as `rspm`, `spm`, and `pm2_5`**:

In [ ]:
# Write your code here
# -------------------------
for region in smooth_monthly['state'].unique():
    # Plot for so2 and no2
    plot = (ggplot(other_measures_df[other_measures_df['state'] == region], aes(x='date', y='value', color='measure')) +
            geom_line() +
            ggtitle(f'Monthly levels of other air quality measures in {region}'))
    plot.show()


AttributeError: 'ggplot' object has no attribute 'show'

By using the simple line chart multiples, you might find lots of project questions, such as

1. Is the $\mathrm{NO}_2$ increase correlated with the increase in respiratory suspended particulate matter (RSPM)?
2. Why does the $\mathrm{SO}_2$ continuously decline over all regions?
3. Does the RSPM increase the incidence of chronic respiratory diseases such as asthma or chronic obstructive pulmonary disease (COPD)?

Now, let's create the choropleth map. In Module 8, students learned how to make a choropleth map using the classic method with predefined geospatial information. However, you can also use the `plotly.express.choropleth` class to easily create the map with additional `update_` methods to give the freedom to customize.

Based on the above additional hint and the following data, which were directly downloaded from the GitHub user content page, you can easily draw the choropleth map.

In [ ]:
%matplotlib inline
import altair as alt
import geopandas as gpd
import plotly.express as px
import plotly.graph_objs as go
from matplotlib import pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
from vega_datasets import data

init_notebook_mode(connected=True)

In [ ]:
# Load the India state boundary
india_states = gpd.read_file(
    "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
)
display(india_states)
india_states.crs

,ST_NM,geometry
0,Arunachal Pradesh,"POLYGON ((95.23392 26.68246, 95.23282 26.70579..."
1,Assam,"POLYGON ((95.19465 27.03132, 95.15008 26.99934..."
2,Chandigarh,"POLYGON ((76.83806 30.75487, 76.83301 30.73887..."
3,Karnataka,"POLYGON ((77.55144 18.29191, 77.57026 18.29252..."
4,Manipur,"POLYGON ((94.67545 25.44561, 94.67293 25.42398..."
5,Meghalaya,"POLYGON ((92.42522 25.02966, 92.40163 25.03363..."
6,Mizoram,"POLYGON ((93.00870 24.41178, 93.02462 24.39255..."
7,Nagaland,"POLYGON ((95.19465 27.03132, 95.19717 27.00183..."
8,Punjab,"POLYGON ((76.77673 30.90429, 76.78433 30.87791..."
9,Rajasthan,"POLYGON ((74.52716 29.94279, 74.51002 29.90819..."


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

According to the Geospatial data `india_states`, the map integrates `Dadra & Nagar` and `Daman & Diu` while both regions have separate air quality observations. To overcome this issue, it is necessary to calculate the simple arithmetic average of two regions if two stations collect observations at the same time.

In [ ]:
# Separate 'Dadra & Nagar Haveli' and 'Daman & Diu' region records from the other regions first
rest_states = monthly.loc[
    ~(monthly.state.isin(["Dadra & Nagar Haveli", "Daman & Diu"]))
].copy()
four_regions = monthly.loc[
    (monthly.state.isin(["Dadra & Nagar Haveli", "Daman & Diu"]))
].copy()

# Calculate group mean
four_regions = (
    four_regions.set_index(["state", "date", "measure"])
    .groupby(["date", "measure"])
    .mean()
    .reset_index()
)
four_regions["state"] = "Dadra and Nagar Haveli and Daman and Diu"

# Now update the data using rest_states and four_regions
monthly = pd.concat([rest_states, four_regions], axis=0)

Then students can easily join the geospatial data `india_states` and its attribute data `monthly`.

In [ ]:
geo_monthly = india_states.rename({"ST_NM": "state"}, axis=1).join(
    monthly.set_index(["state"]), how="right", on=["state"]
)

geo_monthly.head()

,state,geometry,date,measure,value
33.0,Andhra Pradesh,"POLYGON ((84.76474 19.07728, 84.74627 19.04820...",2004-01-01,so2,7.933333
33.0,Andhra Pradesh,"POLYGON ((84.76474 19.07728, 84.74627 19.04820...",2004-02-01,so2,7.577778
33.0,Andhra Pradesh,"POLYGON ((84.76474 19.07728, 84.74627 19.04820...",2004-03-01,so2,7.707018
33.0,Andhra Pradesh,"POLYGON ((84.76474 19.07728, 84.74627 19.04820...",2004-04-01,so2,5.912281
33.0,Andhra Pradesh,"POLYGON ((84.76474 19.07728, 84.74627 19.04820...",2004-05-01,so2,6.754237


**Activity 9**. Create the choropleth map to visualize the India's RSPM(`rspm`) on June 2015 with `Plotly` or `Altair`.

In [ ]:
# Write your code
# ---------------------------
#Filter Data for June 2015:
monthly_june_2015 = monthly[(monthly['date'] == '2015-06-01') & (monthly['measure'] == 'rspm')]

#Merge Data with GeoJSON:
#Merge the filtered data (monthly_june_2015) with india_states GeoJSON based on the state names.
geo_monthly = india_states.rename(columns={"ST_NM": "state"}).merge(
    monthly_june_2015,
    left_on='state',
    right_on='state',
    how='left'
)

#Create the Chloropleth Map with Plotly:
fig = px.choropleth(
    geo_monthly,
    geojson=geo_monthly.geometry,
    locations=geo_monthly.index,
    color='value',  # RSPM values
    hover_name='state',
    projection='mercator',
    title='RSPM Pollution in India (June 2015)',
    color_continuous_scale='Reds',  # Choose your color scale
    range_color=(0, geo_monthly['value'].max())
)

fig.update_geos(fitbounds="locations", visible=False)

fig.update_layout(
    autosize=False,
    width=800,  # Set the width of the figure
    height=600,  # Set the height of the figure
)

fig.show()





---

Next, you will **analyze networks and visualize the results**.


Let's start with analyzing the **traffic between airports**.

In [ ]:
import igraph as ig
import networkx as nx
import pandas as pd
from igraph import Graph, plot

import matplotlib.pyplot as plt
from IPython.display import IFrame
from pyvis import network as net

# flight destinations and counts
flights = pd.read_csv("./data/flights.csv")

**Activity 10**. **Create a data frame** that only has `airport1`, `airport2`, and the `cnt` attributes.

In [ ]:
# -------------------------------------------------------
# Please write your codes in the cell and execute those.
#
flights_subset = flights.loc[:, ['airport1', 'airport2', 'cnt']]

# Create a new DataFrame with only these columns
df_subset = pd.DataFrame(flights_subset)

# Display or use df_subset as needed
print(df_subset.head())

**Activity 11**. **Create a graph from this data frame**, use `directed=False` to make it an **undirected** graph with **Graph.DataFrame** of `igraph`. If you want to use **networkX**, please convert **iGraph** Graph to **networkX** Graph. **Hint**: Use  `to_networkx()` of `NetworkX`.  Please thoroughly read the `igraph` library to use `Graph.DataFrame` with appropriate arguments.

In [ ]:
# -------------------------------------------------------
# Please write your codes in the cell and execute those.
#

# Extract unique vertices
vertices = list(set(df_subset['airport1']).union(set(df_subset['airport2'])))

# Create an empty undirected graph
g = Graph(directed=False)

# Add vertices to the graph
g.add_vertices(vertices)

# Add edges and edge weights from the DataFrame
for index, row in df_subset.iterrows():
    g.add_edge(row['airport1'], row['airport2'], weight=row['cnt'])

# Print summary of the graph
print(g.summary())


In [ ]:
import networkx as nx

unique_airports = pd.concat([df_subset['airport1'], df_subset['airport2']]).unique()
airport_to_id = {airport: i for i, airport in enumerate(unique_airports)}

# Map the airport codes in the DataFrame to their corresponding IDs
df_subset['source'] = df_subset['airport1'].map(airport_to_id)
df_subset['target'] = df_subset['airport2'].map(airport_to_id)

# Create an undirected graph from the DataFrame using igraph
g = ig.Graph.DataFrame(df_subset[['source', 'target', 'cnt']], directed=False)

# Convert the igraph graph to a networkx graph
G = nx.from_pandas_edgelist(df_subset, 'source', 'target', 'cnt')

**Activity 12**. Plot the network with igraph's `plot` function with a **force-directed layout**.

In [ ]:
# -------------------------------------------------------
# Please write your codes in the cell and execute those.
#
# Create a layout for the graph
layout = g.layout_fruchterman_reingold()

# Plot the graph
ig.plot(g, layout=layout)



Now, we will **reduce multiple edges** between vertices by adding all their attributes. There are multiple airlines operating between two airports, **we add their flight counts.**

In [ ]:
# Use Graph.simplify in iGraph
gs = ig.Graph.simplify(g, combine_edges="sum")

# Normalize
gs.es['cnt'] = [cnt / max(gs.es['cnt']) for cnt in gs.es['cnt']]

**Activity 13**. Plot again, this time, **assign the edge weights to `edge.width` parameter.**

In [ ]:
# -------------------------------------------------------
# Please write your codes in the cell and execute those.
#

layout = gs.layout_fruchterman_reingold()

# Plot the graph with edge widths proportional to the 'cnt' attribute
ig.plot(gs, layout=layout, edge_width=gs.es['cnt'])

Now we can see the traffic weighted by the flight counts. Let's change the **size of the vertices by using the traffic**.


We need to **sum up the weights of all the edges for each vertex**.

In [ ]:
# Summing up the edge weights of the adjacent edge for each vertex
gs.vs['traffic'] = ig.Graph.strength(gs, mode='all', weights=gs.es['cnt'])

# normalize
gs.vs['traffic'] = [traffic / max(gs.vs['traffic']) for traffic in gs.vs['traffic']]

**Activity 14**. Plot again, this time, **assign the `gs.vs['traffic']` to the size of the vertices. Make sure to multiply it by a value to make the graph look nice.**

In [ ]:
layout = gs.layout_fruchterman_reingold()

# Plot the graph with vertex sizes proportional to the 'traffic' attribute
# Multiply the 'traffic' attribute by a constant (e.g., 20) to make the vertices visible
ig.plot(gs, layout=layout, vertex_size=[traffic * 30 for traffic in gs.vs['traffic']], edge_width=gs.es['cnt'])

Now we can see that some airports are busier than others, but we don't know their **names**. Let's find out by **removing the vertex shape** and **leaving the vertex label** and use a **font size proportional to the traffic**.


**Activity 15**. Plot again, this time: `vertex.shape="none"` and `vertex.label.cex` should be **proportional to traffic.**

In [ ]:
# -------------------------------------------------------
# Please write your codes in the cell and execute those.
#
# Create a mapping of unique integer IDs to airport codes
id_to_airport = {i: airport for airport, i in airport_to_id.items()}

# Assign the 'name' attribute to the vertices of the graph
gs.vs['name'] = [id_to_airport[v.index] for v in gs.vs]

# Now you can plot the graph with vertex labels
layout = gs.layout_fruchterman_reingold()
ig.plot(gs, layout=layout,
        vertex_shape="none",  # remove the vertex shape
        vertex_label=gs.vs["name"],  # use airport names as labels
        vertex_label_size=[traffic * 30 for traffic in gs.vs['traffic']],  # font size proportional to traffic
        edge_width=gs.es['cnt'])


Let's **get rid of vertices** that do not have much traffic.

In [ ]:
# find them
dv = [i for i, traffic in enumerate(gs.vs['traffic']) if traffic < 0.3]

# Copy the gs before subsetting with the condition above
sub_gs = gs.copy()
# Subset the gs
ig.Graph.delete_vertices(sub_gs, sub_gs.vs[dv])

**Activity 16**. Plot again, this time use a **vertex size and label font proportional to traffic** and make sure to make it look nice.

In [ ]:
# -------------------------------------------------------
# Please write your codes in the cell and execute those.
#
# Create a layout for the graph
layout = sub_gs.layout_fruchterman_reingold()

# Plot the graph with vertex sizes and label font sizes proportional to the 'traffic' attribute
# Multiply the 'traffic' attribute by a constant (e.g., 20 for vertex size and 2 for label font size) to make the vertices and labels visible
ig.plot(sub_gs, layout=layout,
        vertex_shape="none",  # remove the vertex shape
        vertex_size=[traffic * 50 for traffic in sub_gs.vs['traffic']],  # vertex size proportional to traffic
        vertex_label=sub_gs.vs["name"],  # use airport names as labels
        vertex_label_size=[traffic * 30 for traffic in sub_gs.vs['traffic']],  # label font size proportional to traffic
        edge_width=sub_gs.es['cnt'])


### Please save your notebook: File -> Save Notebook (Ctrl+S)

#### **Use the file name format as follows:**

m5-m9_exercise_(_Your #700 number including '700'_).ipynb, **e.g., m5-m9_exercise_700729831.ipynb**.